# Web APIを用いたデータ取得
国会の議事録API
https://kokkai.ndl.go.jp/api.html
を使用してデータを取得する。  
今回は本会議と経済産業委員会の2年分の議事録を取得する。

In [1]:
import urllib
import urllib.request
import json
import time
import glob
import pandas as pd

## API仕様概要と実装内容
国会の議事録APIは検索条件をURLに渡して投げることで国会の議事録を取得することができる。URLの種類は取得単位によっていくつかあるが、今回使用するURLは会議単位で取れる  
https://kokkai.ndl.go.jp/api/meeting?{検索条件}  
を用いることとにする。
今回は検索条件として次の条件を設定する、

|        |                            |
|:-------|:---------------------------|
|日付範囲 |2018−01−01〜2019−12−31(2年分)|
|会議    |本会議or経済産業委員会          |
|議会    |参議院or衆議院                 | 

jsonで出力される。以下のようなURLを投げる。

https://kokkai.ndl.go.jp/api/meeting?from=2018-01-01&until=2019-12-31&nameOfMeeting=本会議 経済産業委員会&nameOfHouse=参議院 衆議院&startRecord=1&recordPacking=json

これをエンコードしたものを投げる

https://kokkai.ndl.go.jp/api/meeting?from%3D2018-01-01%26until%3D2019-12-31%26nameOfMeeting%3D%E6%9C%AC%E4%BC%9A%E8%AD%B0%20%E7%B5%8C%E6%B8%88%E7%94%A3%E6%A5%AD%E5%A7%94%E5%93%A1%E4%BC%9A%26nameOfHouse%3D%E5%8F%82%E8%AD%B0%E9%99%A2%20%E8%A1%86%E8%AD%B0%E9%99%A2%26startRecord%3D1%26recordPacking%3Djson

ヒットした最初の二件しか取得されないので、開始位置をずらしながらURLを投げて全件取得する(全件の件数はnumberOfRecordsに格納される)。

|        |              |
|:-------|:-------------|
|1回目   |startRecord=1 |
|2回目   |startRecord=3 |
|3回目   |startRecord=5 | 
|....   |....          | 

In [2]:
start = '2018-01-01'
end = '2019-12-31'
meetings = ['本会議', '経済産業委員会']
houses = ['参議院', '衆議院']

In [3]:
def create_url(start, end, meetings, houses, start_record=1):
    meetings = ' '.join(meetings)
    houses = ' '.join(houses)
    url = 'from={}&until={}&nameOfMeeting={}&nameOfHouse={}&startRecord={}&recordPacking=json'.\
    format(start, end, meetings, houses, start_record)
    url = 'https://kokkai.ndl.go.jp/api/meeting?{}'.format(urllib.parse.quote(url))
    return url

In [4]:
def record_nums(json_data):
    '''
    numberOfRecords: 総結果件数
    numberOfReturn: 返戻件数
    startRecord: 開始位置
    nextRecordPosition: 次開始位置（※存在する場合のみ）
    '''
    return json_data['numberOfRecords'], \
           json_data['numberOfReturn'], \
           json_data['startRecord'], \
           json_data['nextRecordPosition']

In [5]:
url = create_url(start, end,  meetings, houses)
try:
    res = urllib.request.Request(url) #urlリクエスト
    # url リクエストを開く
    with urllib.request.urlopen(res) as r:
        # jsonとして開く
        json_data = json.load(r)
        # jsonのヘッダ部分のみを取得する
        records_num, return_num, start_record, next_position = record_nums(json_data)
        print('records_num:{} startRecord:{}'.format(records_num, start_record))
        while next_position is not None:
            # 会議単位で保管するためにmeetingRecordごとに保管する
            for content in json_data['meetingRecord']:
                # とりたい会議以外が取れる場合があるので、その場合は除く
                if content['nameOfMeeting'] not in meetings:
                    continue
                
                # 参議院と衆議院でファイル名を分ける
                file_path = '../input/{}_sanin_{}.json' if content['nameOfHouse'] == '参議院' \
                       else '../input/{}_syuin_{}.json'
                file_path = file_path.format(content['date'], content['nameOfMeeting'])
                # jsonを保管する
                with open(file_path, 'w') as f:
                    content = json.dumps(content)
                    f.write(content)
                    
                time.sleep(2)
            
            # 次のurlを作成し、リクエストを投げる
            url = create_url(start, end,  meetings, houses, next_position)
            res = urllib.request.Request(url)
            with urllib.request.urlopen(res) as r:
                json_data = json.load(r)
                records_num, return_num, start_record, next_position = record_nums(json_data)
                print('records_num:{} startRecord:{}'.format(records_num, start_record))
        else:
            print('done!')
                    
except urllib.error.HTTPError as err:
    print(err.code)
except urllib.error.URLError as err:
    print(err.reason)

records_num:292 startRecord:1
records_num:292 startRecord:3
records_num:292 startRecord:5
records_num:292 startRecord:7
records_num:292 startRecord:9
records_num:292 startRecord:11
records_num:292 startRecord:13
records_num:292 startRecord:15
records_num:292 startRecord:17
records_num:292 startRecord:19
records_num:292 startRecord:21
records_num:292 startRecord:23
records_num:292 startRecord:25
records_num:292 startRecord:27
records_num:292 startRecord:29
records_num:292 startRecord:31
records_num:292 startRecord:33
records_num:292 startRecord:35
records_num:292 startRecord:37
records_num:292 startRecord:39
records_num:292 startRecord:41
records_num:292 startRecord:43
records_num:292 startRecord:45
records_num:292 startRecord:47
records_num:292 startRecord:49
records_num:292 startRecord:51
records_num:292 startRecord:53
records_num:292 startRecord:55
records_num:292 startRecord:57
records_num:292 startRecord:59
records_num:292 startRecord:61
records_num:292 startRecord:63
records_num:2

# ファイル加工
今後のためcsv形式に変換しておく。以下の形式のファイルを作成する。
ファイル形式は以下の通り。

|date|house       |meeting|speech_order|text|
|:---|:-----------|:------|:-----------|:---|
|日付 |参議院or衆議院|委員会名|発言順序 　 　|発言内容|

In [5]:
file_paths = glob.glob('../input/*.json')

In [6]:
kokkai_list = []
for file_path in file_paths:
    with open(file_path, 'r') as f:
        json_data = json.load(f)
        date = json_data['date']
        house = json_data['nameOfHouse']
        meeting = json_data['nameOfMeeting']
        texts = [[date, house, meeting, x['speechOrder'], x['speech']] for x in json_data['speechRecord']]
        kokkai_list.extend(texts)

In [7]:
df_kokkai = pd.DataFrame(kokkai_list, columns=['date', 'house', 'meeting', 'speech_order', 'text'])
df_kokkai.to_csv('../data/kokkai.csv', index=False)